In [2]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays 
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
# include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [3]:
Params = get_default_params_PLD()
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
# num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [6]:
S_train_list = Params["S_train_all"] # training data size
is_ridge = Params["is_ridge"] # whether to use ridge regression
S_train_list = [500] # for quick testing
instances = 10 # for quick testing

10

In [7]:
Random.seed!(seed)
is_Wang_Qi_Shen = true;
is_same_util_para = true;
if is_Wang_Qi_Shen
    project_dir = "Mixex_MNL=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    project_dir = "Model_Mis_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")

data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Codes/Data/Product_Line_Design/Mixex_MNL=3_N_x=8_N_u=1_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/


In [8]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

## No Model Mis

In [9]:
function Estimation_Process(S_train_list,lambda,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            nu0_true,nu_true = compute_w(theta_true,Z_test[1,:])  
            nu_all_true = [nu0_true;nu_true]
            Input_Data_this["nu_true"] = nu_all_true

            # ******** Estimation *************
            theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Asorrtment_train,is_ridge, lambda)
            nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  
            nu_all_hat = [nu0_hat;nu_hat]
            Input_Data_this["theta_hat"] = theta_hat
            Input_Data_this["nu_hat"] = nu_all_hat

            Input_Data["ins=$(ins)"] = Input_Data_this
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "Input_Data_S=$(S_train)_lambda=$lambda.jld2"), Input_Data);
    end
end

Estimation_Process (generic function with 1 method)

In [13]:
lambda = 0.001
is_ridge = false
Estimation_Process([10000],lambda,instances)

********** S_train = 10000 **********
******* ins = 1*********
******* ins = 2*********
******* ins = 3*********
******* ins = 4*********
******* ins = 5*********
******* ins = 6*********
******* ins = 7*********
******* ins = 8*********
******* ins = 9*********
******* ins = 10*********


In [15]:
S_train = 10000
Input_Data_S = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$lambda.jld2"));

In [16]:
for ins in 1:instances
    println("True Nu = ",round.(Input_Data_S["ins=$ins"]["nu_true"],digits=4))
    println("Esti Nu = ",round.(Input_Data_S["ins=$ins"]["nu_hat"],digits=4))
    println("----------------------------")
end

True Nu = [0.1125, 0.5678, -1.7343, -1.4321, -1.2869, 0.275, -0.3388, -0.2617, -1.8003]
Esti Nu = [0.1592, 0.5827, -1.7509, -1.4963, -1.3563, 0.2441, -0.3295, -0.2906, -1.8034]
----------------------------
True Nu = [1.3047, -0.5654, -0.2781, -0.5443, -0.4206, -0.2498, -0.4879, -0.1002, -0.2248]
Esti Nu = [1.3089, -0.5444, -0.3013, -0.5406, -0.4086, -0.2316, -0.4734, -0.1107, -0.2011]
----------------------------
True Nu = [-0.4564, 1.1086, -2.4293, -1.8558, -1.7004, 0.5255, -0.2677, -0.3388, -2.5521]
Esti Nu = [-0.2806, 1.0498, -2.4795, -1.9335, -1.8446, 0.5173, -0.2928, -0.4224, -2.5461]
----------------------------
True Nu = [-0.3744, 1.0306, -2.3291, -1.7948, -1.6408, 0.4894, -0.2779, -0.3276, -2.4437]
Esti Nu = [-0.4371, 1.0986, -2.3657, -1.7195, -1.6109, 0.5896, -0.3596, -0.3159, -2.6168]
----------------------------
True Nu = [-0.3868, 1.0424, -2.3443, -1.804, -1.6498, 0.4948, -0.2764, -0.3293, -2.4601]
Esti Nu = [-0.4363, 1.0628, -2.264, -1.6601, -1.5809, 0.4915, -0.2409, -0.40

### Generate Utility Coef for diff segment

In [89]:
function Generate_Wang_Qi_Max_True_Paras_Multi_Class(coef_Params, theta_true_Fixed, num_c)
    theta_true_Fixed_all = Dict()
    for c in 1:num_c
        alpha_c = deepcopy(theta_true_Fixed.alpha)
        alpha_c[1:c*2] = rand(Uniform(coef_Params.alp_lb, coef_Params.alp_ub), 2*c)
        theta_true_Fixed_c = (alpha0 = theta_true_Fixed.alpha0,
                            alpha=alpha_c,
                            beta=theta_true_Fixed.beta,
                            A=theta_true_Fixed.A)
        theta_true_Fixed_all["class=$(c)"] = theta_true_Fixed_c
    end
    return theta_true_Fixed_all
end

Generate_Wang_Qi_Max_True_Paras_Multi_Class (generic function with 3 methods)

In [93]:
function Generate_Wang_Qi_Max_True_Data_Multi_Class(N_x, N_u, n_sample, m,theta_true,u_lb,u_ub)
    # 初始化存储
    X = Vector{Matrix{Float64}}(undef, n_sample); # n 个 m×d 矩阵
    Z = Matrix{Float64}(undef, n_sample, N_u);      # n×p 矩阵
    Y = Vector{Int}(undef, n_sample);             # n 维向量

    for i in 1:n_sample
        z_i = rand(Uniform(u_lb, u_ub), N_u)
        Z[i, :] = z_i

        X_i = zeros(m, N_x)
        for j in 1:m
            # X_i[j, :] = rand(Uniform(0.0, 1.0), d)
            X_i[j, :] = rand(0.0:1.0, N_x)
        end
        X[i] = X_i

        # --- 计算选择概率 ---
        # 根据公式 (2.1): Pz(x; θ*) = exp(Uz(x)) / (V0 + exp(Uz(x)))
        # 论文中 V0 = exp(U0) = 1 (默认选项效用权重归一化为1)
        V0 = 1.0
        utilities = zeros(m)
        for j in 1:m
            x_ij = X_i[j, :] # 第 j 个产品的设计
            # 计算效用 Uz(x_ij) = α₀* + <α*, x_ij> + <β*, z_i> + x_ij^T * A* * z_i
            utility = theta_true.alpha0 +
                        dot(theta_true.alpha, x_ij) +
                        dot(theta_true.beta, z_i) +
                        dot(x_ij, theta_true.A * z_i) # x_ij^T * A* * z_i
            utilities[j] = utility
        end

        # 计算分子 exp(Uz(x_ij))
        exp_utilities = exp.(utilities)
        # 计算分母 (V0 + sum(exp(Uz(x_il))))
        denominator = V0 + sum(exp_utilities)

        # 计算选择每个产品 j 的概率
        prob_choose_product = exp_utilities ./ denominator
        # 计算选择默认选项 (索引 0) 的概率
        prob_choose_default = V0 / denominator

        # 构建完整的概率向量 [P(选择默认), P(选择产品1), ..., P(选择产品m_actual)]
        choice_probs = vcat(prob_choose_default, prob_choose_product)
        # 选择结果: 0 表示默认选项, 1 表示第一个产品, ..., m_actual 表示第 m_actual 个产品
        y_i = sample(0:m, Weights(choice_probs))
        Y[i] = y_i
    end
    asorrtment_train = Array{Vector{Int64}}(undef,n_sample)
    for s in 1:n_sample
        asorrtment_train[s] = collect(1:N_Max)
    end
    return X,Y,Z,asorrtment_train
end

Generate_Wang_Qi_Max_True_Data_Multi_Class (generic function with 1 method)

In [90]:
num_c = 2
theta_true_Fixed_all = Generate_Wang_Qi_Max_True_Paras_Multi_Class(coef_para_Input, theta_true_Fixed, num_c)

Dict{Any, Any} with 2 entries:
  "class=2" => (alpha0 = 1.00225, alpha = [-0.576795, -0.172876, -0.426319, -0.…
  "class=1" => (alpha0 = 1.00225, alpha = [-0.362344, -0.845952, -0.769507, -0.…

In [97]:
n_samples = [10000, 10000] # number of samples per class
lowers = -0.1*ones(num_c)
uppers = 0.1*ones(num_c)


Z_test_arr = rand(Uniform(lowers[1], uppers[1]), N_u)


X_train_all = Dict()
Y_train_all = Dict()
Z_train_all = Dict()
Assort_train_all = Dict()
for c in 1:num_c
    u_lb_c = lowers[c]
    u_ub_c = uppers[c]
    n_sample_c = n_samples[c]
    theta_true_c = theta_true_Fixed_all["class=$(c)"]
    nu0_true,nu_true = compute_w(theta_true_c,Z_test_arr)  
    nu_all_true = [nu0_true;nu_true]
    println("Class $(c) True Nu = ",round.(nu_all_true,digits=4))

    X_c,Y_c,Z_c,Assort_c = Generate_Wang_Qi_Max_True_Data_Multi_Class(N_x, N_u, n_sample_c, N_Max,theta_true_c,u_lb_c,u_ub_c)
    theta_hat = Estimation_This(N_Max,N_x,N_u,Y_c,X_c,Z_c, Assort_c,is_ridge, lambda)
    nu0_hat,nu_hat = compute_w(theta_hat,Z_test_arr)  
    nu_all_hat = [nu0_hat;nu_hat]
    println("Class $(c) Esti Nu = ",round.(nu_all_hat,digits=4))
    
    X_train_all["class=$(c)"] = X_c
    Y_train_all["class=$(c)"] = Y_c
    Z_train_all["class=$(c)"] = Z_c
    Assort_train_all["class=$(c)"] = Assort_c
end
X_train = vcat([X_train_all["class=$(c)"] for c in 1:num_c]...);
Y_train = vcat([Y_train_all["class=$(c)"] for c in 1:num_c]...);
Z_train = vcat([Z_train_all["class=$(c)"] for c in 1:num_c]...);
Assort_train = vcat([Assort_train_all["class=$(c)"] for c in 1:num_c]...);

Class 1 True Nu = [0.9752, -0.3366, -0.879, -0.7896, -0.66, -0.1047, -0.4467, -0.1449, -0.6602]
Class 1 Esti Nu = [0.9324, -0.3337, -0.9178, -0.7602, -0.5927, -0.0929, -0.4295, -0.0836, -0.6771]
Class 2 True Nu = [0.9752, -0.5511, -0.2059, -0.4465, -0.0312, -0.1047, -0.4467, -0.1449, -0.6602]
Class 2 Esti Nu = [1.0251, -0.5789, -0.2189, -0.4397, -0.021, -0.037, -0.4652, -0.168, -0.6638]


In [98]:
theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Assort_train,is_ridge, lambda)
nu0_hat,nu_hat = compute_w(theta_hat,Z_test_arr)  
nu_all_hat = [nu0_hat;nu_hat]
println("Pooling Esti Nu = ",round.(nu_all_hat,digits=4))

Pooling Esti Nu = [0.9422, -0.4634, -0.5334, -0.5809, -0.2808, -0.0635, -0.4446, -0.1279, -0.6667]
